<a href="https://colab.research.google.com/github/sam-thurman/asl_alphabet_image_classification/blob/master/exploratory_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import scipy as math
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
# from PIL import Image

from sklearn.model_selection import train_test_split
import keras

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from keras.models import Model, Sequential
from keras import layers
from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras import optimizers
from keras.optimizers import Adam

import sys
import os


In [0]:
# mount google drive
from google.colab import drive
# drive.mount('/content/drive/My Drive/data')
drive.mount('/content/drive')#, force_remount=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# change directory into drive
%cd drive/My\ Drive/


/content/drive/My Drive


In [0]:
%ls

 Accounts.gdoc
 asl_alphabet_image_classification/
'bb logo v2 (05:29:19).pdf'
'Behavioral quizz write-up 301.gdoc'
'Capstone MVP presentation.gslides'
'ceative resume.gdoc'
'CF 301 reading journal.gdoc'
 Codefellows/
'(codefellows)Answers to Common Behavioral Questions.gdoc'
 communication.mp3
 copy_base__inceptionv3.h5
 copy_base_model.h5
 copy_model1.h5
 copy_model2.h5
 data/
 Fin:BobbyBrooka_AsIShould.mp3
 Fin_Mischief.mp3
'Fin_Wreckage 3.mp3'
'Flatiron Capstone Proposal.gdoc'
'flatiron notes'/
'Getting started.pdf'
"hot n' cold beat RW.mp3"
'ice man (concept not finished).mp3'
'Image 11-18-19 at 11.12 AM.JPG'
'in my ceiling.mp3'
'in my ceiling (w VERY ROUGH voc concept).mp3'
'Kaia_RedWine_BetweenUs 2.mp3'
 Kaia_RedWine_BetweenUs.mp3
'mischief (1).mp3'
 mischief.mp3
'Norwood_TollFree[Master].mp3'
'nothing nowhere.mp3'
'pain on your thigh'/
'pain on your thigh 1-Drum Rack.mp3'
'pain on your thigh 2-Addictive Keys.mp3'
'pain on your thigh 3-Addictive Keys.mp3'
'pain on your thigh 4-S

In [0]:
# path for files inside content/drive/My Drive notebooks folder
# train_path = os.path.join('data/asl_alphabet_train')
# validation_path = os.path.join('data/asl_alphabet_validation')
train_path = 'data/asl_alphabet_train'
validation_path = 'data/asl_alphabet_validation'

In [0]:
# image generator instance
random_seed = 42
datagen = keras.preprocessing.image.ImageDataGenerator(
                rescale=1./255,
                validation_split=0.3) 


In [0]:
image_size = 150
batch_size = 32

print('done')
train_generator = datagen.flow_from_directory(
                train_path,  #directory for training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                class_mode='categorical',
                color_mode='grayscale',
                shuffle=True,
                subset='training')
print('done')
test_generator = datagen.flow_from_directory(
                train_path,  # same directory for testing images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                class_mode='categorical',
                color_mode='grayscale',
                shuffle=True,
                subset='validation')
print('done')



done
Found 60991 images belonging to 29 classes.
done
Found 26134 images belonging to 29 classes.
done


In [0]:
test_generator.class_indices

{'A': 0,
 'B': 1,
 'C': 2,
 'D': 3,
 'E': 4,
 'F': 5,
 'G': 6,
 'H': 7,
 'I': 8,
 'J': 9,
 'K': 10,
 'L': 11,
 'M': 12,
 'N': 13,
 'O': 14,
 'P': 15,
 'Q': 16,
 'R': 17,
 'S': 18,
 'T': 19,
 'U': 20,
 'V': 21,
 'W': 22,
 'X': 23,
 'Y': 24,
 'Z': 25,
 'del': 26,
 'nothing': 27,
 'space': 28}

In [0]:
image_size = 200
batch_size = 32
valgen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255) 
val_generator = valgen.flow_from_directory(
                validation_path,  # directory for validation images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                class_mode='categorical',
                color_mode='grayscale',
                shuffle=False)

Found 30 images belonging to 29 classes.


In [0]:
nb_classes = 29

# initialize CNN
model = Sequential()

# 1st conv layer
model.add(Conv2D(32,(3,3), padding='same', input_shape=(200, 200,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 2nd conv layer
model.add(Conv2D(64,(5,5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 3rd conv layer
model.add(Conv2D(128,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 4th conv layer
# model.add(Conv2D(128,(3,3), padding='same'))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# flattening
model.add(Flatten())

# 1st fully connected layer
# model.add(Dense(64))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(Dropout(0.25))

# 2nd fully connected layer
model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(nb_classes, activation='softmax'))



In [0]:
optimizer = Adam(lr=0.01)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
hisotry = model.fit_generator(test_generator, epochs=5)

Epoch 1/5
545/545 [==============================] - 6605s 12s/step - loss: 0.9801 - accuracy: 0.6808
Epoch 2/5
545/545 [==============================] - 119s 219ms/step - loss: 0.3006 - accuracy: 0.8991
Epoch 3/5
545/545 [==============================] - 119s 219ms/step - loss: 0.1747 - accuracy: 0.9401
Epoch 4/5
545/545 [==============================] - 119s 219ms/step - loss: 0.1259 - accuracy: 0.9569
Epoch 5/5
545/545 [==============================] - 120s 219ms/step - loss: 0.1063 - accuracy: 0.9651


In [0]:
model.save('base_model.h5')
del model

In [0]:
%ls

 Accounts.gdoc
 asl_alphabet_image_classification/
 base_model.h5
'bb logo v2 (05:29:19).pdf'
'Behavioral quizz write-up 301.gdoc'
'Capstone MVP presentation.gslides'
'ceative resume.gdoc'
'CF 301 reading journal.gdoc'
 Codefellows/
'(codefellows)Answers to Common Behavioral Questions.gdoc'
 communication.mp3
 data/
 Fin:BobbyBrooka_AsIShould.mp3
 Fin_Mischief.mp3
'Fin_Wreckage 3.mp3'
'Flatiron Capstone Proposal.gdoc'
'flatiron notes'/
'Getting started.pdf'
"hot n' cold beat RW.mp3"
'ice man (concept not finished).mp3'
'Image 11-18-19 at 11.12 AM.JPG'
'in my ceiling.mp3'
'in my ceiling (w VERY ROUGH voc concept).mp3'
'Kaia_RedWine_BetweenUs 2.mp3'
 Kaia_RedWine_BetweenUs.mp3
'mischief (1).mp3'
 mischief.mp3
'Norwood_TollFree[Master].mp3'
'nothing nowhere.mp3'
'pain on your thigh'/
'pain on your thigh 1-Drum Rack.mp3'
'pain on your thigh 2-Addictive Keys.mp3'
'pain on your thigh 3-Addictive Keys.mp3'
'pain on your thigh 4-Serum.mp3'
'pain on your thigh 5-Serum.mp3'
'pain on your thigh 6

In [0]:
from keras.models import load_model
model = load_model('asl_alphabet_image_classification/models/base_model.h5')

In [0]:
y_pred = model.predict_generator(val_generator)

In [0]:
y_pred.argmax(-1)

array([11,  1,  2,  3,  3,  5,  9,  7,  8,  9, 10, 14, 12, 13, 14, 15, 15,
       17, 12, 19, 20, 21, 21, 23, 24, 25, 26, 27, 27, 28])

In [0]:
y_true = val_generator.labels

In [0]:
pred_indices = []
for pred in y_pred:
  pred_indices.append(np.argmax(pred))

In [0]:
print(list(y_pred.argmax(-1)))
print(list(y_true))

[11, 1, 2, 3, 3, 5, 9, 7, 8, 9, 10, 14, 12, 13, 14, 15, 15, 17, 12, 19, 20, 21, 21, 23, 24, 25, 26, 27, 27, 28]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 27, 28]


In [0]:
print(val_generator.class_indices)

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25, 'nothing': 26, 'space': 27}


In [0]:
nb_classes = 29

# initialize CNN
model1 = Sequential()

# 1st conv layer
model1.add(Conv2D(32,(3,3), padding='same', input_shape=(image_size, image_size,1)))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Dropout(0.25))

# 2nd conv layer
model1.add(Conv2D(64,(5,5), padding='same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Dropout(0.25))

# 3rd conv layer
model1.add(Conv2D(128,(3,3), padding='same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Dropout(0.25))

# 4th conv layer
model1.add(Conv2D(128,(3,3), padding='same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Dropout(0.25))

# flattening
model1.add(Flatten())

# 1st fully connected layer
model1.add(Dense(64))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(Dropout(0.25))

# 2nd fully connected layer
model1.add(Dense(128))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(Dropout(0.25))

model1.add(Dense(nb_classes, activation='softmax'))



In [0]:
optimizer = Adam(lr=0.01)
model1.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
hisotry1 = model1.fit_generator(train_generator, epochs=5)

Epoch 1/5
 417/1906 [=====>........................] - ETA: 4:30:08 - loss: 2.4688 - accuracy: 0.2516

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 115 could not be retrieved. It could be because a worker has died.
  UserWarning)


1287/1906 [===================>..........] - ETA: 1:53:13 - loss: 1.6598 - accuracy: 0.4669

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 747 could not be retrieved. It could be because a worker has died.
  UserWarning)


1906/1906 [==============================] - 20529s 11s/step - loss: 1.3884 - accuracy: 0.5495
Epoch 2/5
1906/1906 [==============================] - 908s 476ms/step - loss: 0.5454 - accuracy: 0.8166
Epoch 3/5
1906/1906 [==============================] - 866s 454ms/step - loss: 0.3524 - accuracy: 0.8831
Epoch 4/5
1906/1906 [==============================] - 892s 468ms/step - loss: 0.2619 - accuracy: 0.9144
Epoch 5/5
1906/1906 [==============================] - 889s 467ms/step - loss: 0.2090 - accuracy: 0.9314


In [0]:
model1.save('model1.h5')


In [0]:
%ls

 Accounts.gdoc
 asl_alphabet_image_classification/
'bb logo v2 (05:29:19).pdf'
'Behavioral quizz write-up 301.gdoc'
'Capstone MVP presentation.gslides'
'ceative resume.gdoc'
'CF 301 reading journal.gdoc'
 Codefellows/
'(codefellows)Answers to Common Behavioral Questions.gdoc'
 communication.mp3
 copy_base__inceptionv3.h5
 data/
 Fin:BobbyBrooka_AsIShould.mp3
 Fin_Mischief.mp3
'Fin_Wreckage 3.mp3'
'Flatiron Capstone Proposal.gdoc'
'flatiron notes'/
'Getting started.pdf'
"hot n' cold beat RW.mp3"
'ice man (concept not finished).mp3'
'Image 11-18-19 at 11.12 AM.JPG'
'in my ceiling.mp3'
'in my ceiling (w VERY ROUGH voc concept).mp3'
'Kaia_RedWine_BetweenUs 2.mp3'
 Kaia_RedWine_BetweenUs.mp3
'mischief (1).mp3'
 mischief.mp3
 model1.h5
'Norwood_TollFree[Master].mp3'
'nothing nowhere.mp3'
'pain on your thigh'/
'pain on your thigh 1-Drum Rack.mp3'
'pain on your thigh 2-Addictive Keys.mp3'
'pain on your thigh 3-Addictive Keys.mp3'
'pain on your thigh 4-Serum.mp3'
'pain on your thigh 5-Serum.mp3

In [0]:
del model1

950877